# Protocols

In [34]:
import math
from typing import Union, Iterator, MutableSequence, Protocol, runtime_checkable, Any
from random import shuffle



## Tension Between Typing Systems


Consider an automated lunch shop's digital menu system:
* The restaurant has a variety of entries that are "splittable," meaning you can get a half order.
* Entries like deli sandwiches, wraps, and soups can be split.
* Entries like drinks and hamburgers cannot be split.


In [1]:
# We can represent some entries of the last restaurant description as follows

class BLTSandwich:
    def __init__(self):
        self.cost = 6.95
        self.name = 'BLT'
        # This class handles a fully constructed BLT sandwich
        # ...

    def split_in_half(self) -> tuple['BLTSandwich', 'BLTSandwich']:
        # Instructions for how to split a sandwich in half
        # Cut along diagonal, wrap separately, etc.
        # Return two sandwiches in return
        return

class Chili:
    def __init__(self):
        self.cost = 4.95
        self.name = 'Chili'
        # This class handles a fully loaded chili
        # ...

    def split_in_half(self) -> tuple['Chili', 'Chili']:
        # Instructions for how to split chili in half
        # Ladle into new container, add toppings
        # Return two cups of chili in return
        # ...
        return 

class BaconCheeseburger:
    def __init__(self):
        self.cost = 11.95
        self.name = 'Bacon Cheeseburger'
        # This class handles a delicious Bacon Cheeseburger
        # ...
        # NOTE! no split_in_half method


In [5]:
# The split method might look something like this

def split_dish(dish):
    dishes = dish.split_in_half()
    assert len(dishes) == 2
    for half_dish in dishes:
        half_dish.cost = math.ceil(half_dish.cost) / 2
        half_dish.name = "1/2 " + half_dish.name
    return dishes


In [7]:
# But what about the type signature of the last function?

"""
def split_dish(dish: ???) -> ???:
    return
"""

'\ndef split_dish(dish: ???) -> ???:\n    return\n'

In [10]:
# We can use Any, but this conveys no intent to future developers

def split_dish(dish: Any) -> Any:
    return

In [13]:
# We can use a Union, but we are hardcoding classes into the type signature

def split_dish(dish: Union[BLTSandwich, Chili]):
    """
    Every time somebody needs to add a class that can be splittable, 
    they have to remember to update this function
    """
    return

### Using Inheritance


In [14]:
# We can create a base class with the methods we want to implement,
# and have the entries inherit from that base class

class Splittable:
    def __init__(self, cost, name):
        self.cost = cost
        self.name = name

    def split_in_half(self) -> tuple['Splittable', 'Splittable']:
        raise NotImplementedError("Must implement split in half")

class BLTSandwich(Splittable):
    pass

class Chili(Splittable):
    pass


In [15]:
# And the signature of split_dish() will be as follows

def split_dish(dish: Splittable) -> tuple[Splittable, Splittable]:
    return


But this doesn't scale when the number of classes grows

### Using Mixins

In [16]:
# We can add custom behavior to a class without an inheritance hierarchy with mixins.

class Shareable:
    pass

class PickUppable:
    pass

class Substitutable:
    pass

class BLTSandwich(Shareable, PickUppable, Substitutable, Splittable):
    """
    If we want my BLTSandwich to be Shareable, PickUppable, Substitutable, and Splittable, 
    then we don't have to modify anything else besides BLTSandwich.
    """
    pass

But changing existing classes just for the sake of typechecking feels very unpythonic. It should be a better way.

### Using Protocols

In [23]:
# As an example of a protocol, the iterator protocol is a set of behaviors that objects may implement. 
# If an object implements these behaviors, you can loop over the object

class ShuffleIterator:
    def __init__(self, sequence: MutableSequence):
        self.sequence = list(sequence)
        shuffle(self.sequence)

    def __iter__(self):
        return self

    def __next__(self):
        if not self.sequence:
            raise StopIteration
        return self.sequence.pop(0)


In [27]:
iterator: Iterator = ShuffleIterator([1, 2, 3, 4])
for num in iterator:
    print(num)


2
4
3
1


In [29]:
# The last class implements built-in methods (__iter__ and __next__).
# We can define a custom protocol, with arbitrary methods, as follows

class Splittable(Protocol):
    cost: int
    name: str

    def split_in_half(self) -> tuple['Splittable', 'Splittable']:
        """ No implementation needed """
        return


In [30]:
# To have the BLTSandwich be splittable, we just implement the split_in_half() method. 
# There is no subclassing needed

class BLTSandwich:
    def __init__(self):
        self.cost = 6.95
        self.name = 'BLT'
        # This class handles a fully constructed BLT sandwich
        # ...
    
    def split_in_half(self) -> tuple['BLTSandwich', 'BLTSandwich']:
        # Instructions for how to split a sandwich in half
        # Cut along diagonal, wrap separately, etc.
        # Return two sandwiches in return
        return


In [31]:
# And the signature of split_dish() will be as follows

def split_dish(order: Splittable) -> tuple[Splittable, Splittable]:
    return 

With protocols, we can simplify class hierarchies immensely

## Advanced Usage of Protocols

### Composite Protocols


What if most of the entries are Splittable, Shareable, Substitutable, and PickUppable?

In [32]:
# We can write a type alias, but this will match anything that satisfies 
# at least one protocol, not all four:

StandardLunchEntry = Union[Splittable, Shareable, Substitutable, PickUppable]



In [33]:
# We need to match all 4 protocols, so we  need to use a composite protocol:

class Shareable(Protocol):
    pass

class Substitutable(Protocol):
    pass

class PickUppable(Protocol):
    pass

class StandardLunchEntry(Splittable, Shareable, Substitutable, PickUppable, Protocol):
    """
    # There's no need to explicitly subclass from the protocol, we do so here for clarity's sake
    """
    pass

class BLTSandwich(StandardLunchEntry):
    # ... snip ..
    pass

### Runtime Checkable Protocols


In [37]:
# We can make protocols runtime-checkable (can be used as arg of isinstance() for example) 
# with the 'runtime_checkable' decorator, as follows

@runtime_checkable
class Splittable(Protocol):
    cost: int
    name: str

    def split_in_half(self) -> tuple['Splittable', 'Splittable']:
        #...
        return

class BLTSandwich:
    def __init__(self):
        self.cost = 6.95
        self.name = 'BLT'
        # This class handles a fully constructed BLT sandwich
        # ...
    
    def split_in_half(self) -> tuple['BLTSandwich', 'BLTSandwich']:
        # Instructions for how to split a sandwich in half
        # Cut along diagonal, wrap separately, etc.
        # Return two sandwiches in return
        return


assert isinstance(BLTSandwich(), Splittable)
